## Predict climbing strength:  "The V grade you can send 90-100% of routes" (Any information)

## Preprocessing

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler


(654,) (654, 55)


In [89]:
def formatCode(csvFile):
    Data = pd.read_csv(csvFile)
    y = Data['Hardest_V_Grade_ever_climbed_']
    X = Data.drop(columns=['The_V_grade_you_can_send_90-100%_of_routes_', 'Hardest_V_Grade_ever_climbed_', 
                        'Hardest_V_Grade_climbed_in_the_Last_3_months']) # remove target and correlated features
    X.columns = X.columns.str.replace(r'[^\w\s]', '_', regex=True)
    X.columns = X.columns.str.replace(r' ', '_', regex=True)
    print(y.shape, X.shape)
    return X, y

In [90]:
from sklearn.model_selection import train_test_split
(X, y) = formatCode("formatted_data/formatted_data.csv")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
from sklearn.linear_model import SGDRegressor, Lasso, Ridge, ARDRegression, ElasticNet, HuberRegressor
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, mutual_info_regression, r_regression, f_regression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

In [55]:
import sys, os

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

## Try with LGBM Regressor (Gradient boosting)

In [ ]:
pipe = Pipeline([('select', SelectKBest()),('sc', StandardScaler()), ('reg', lgb.LGBMRegressor(num_leaves=31))])
param_grid = {'select__score_func': [mutual_info_regression, f_regression],
                'select__k': [10, 15, 20, 25, 30, 55],
                'reg__learning_rate': [0.1, 0.25, 0.5],
                'reg__n_estimators': [50, 100, 200],
              }
     
grid_search = GridSearchCV(pipe, param_grid, cv=5, verbose=0)

In [70]:
blockPrint()
grid_search.fit(X_train, y_train)
enablePrint()

c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserW

In [71]:
grid_search.best_params_

{'reg__learning_rate': 0.1,
 'reg__n_estimators': 50,
 'reg__num_leaves': 20,
 'select__k': 25,
 'select__score_func': <function sklearn.feature_selection._univariate_selection.f_regression(X, y, *, center=True, force_finite=True)>}

In [72]:
grid_search.best_score_

np.float64(0.4125690148009792)

In [57]:
# making it more specific
pipe2 = Pipeline([('select', SelectKBest()),('sc', StandardScaler()), ('reg', lgb.LGBMRegressor(num_leaves=31))])
param_grid2 = {'select__score_func': [mutual_info_regression, f_regression],
              'select__k': [21, 23, 25, 27, 29],
              'reg__learning_rate': [0.05, 0.1, 0.15],
              'reg__n_estimators': [25, 50, 75],
              }
     
grid_search2 = GridSearchCV(pipe2, param_grid2, cv=5, verbose=0)
blockPrint()
grid_search2.fit(X_train, y_train)
enablePrint()

c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserW

In [ ]:
grid_search2.best_params_

In [ ]:
grid_search2.best_score_ #slightly better

np.float64(0.4248225665183706)

## Try with ElasticNet

In [73]:
pipe3 = Pipeline([('select', SelectKBest()),('sc', StandardScaler()), ('reg', ElasticNet(random_state=42, max_iter=100000))], memory=None)
     

param_grid3 = {'select__score_func': [mutual_info_regression, f_regression],
              'select__k': [10, 15, 20, 25, 30, 40],
              'reg__alpha': [0.1, 0.25, 0.5, 0.75, 1],
              'reg__l1_ratio': [0, 0.25, 0.5, 0.75, 1]
              }
     
grid_search3 = GridSearchCV(pipe3, param_grid3, cv=5)


In [74]:
grid_search3.fit(X_train, y_train)

c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.125e+02, tolerance: 2.249e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.043e+02, tolerance: 2.205e-01 Linear regression models with null weight for the l1 regularization term are more efficiently 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('select', SelectKBest()),
                                       ('sc', StandardScaler()),
                                       ('reg',
                                        ElasticNet(max_iter=100000,
                                                   random_state=42))]),
             param_grid={'reg__alpha': [0.1, 0.25, 0.5, 0.75, 1],
                         'reg__l1_ratio': [0, 0.25, 0.5, 0.75, 1],
                         'select__k': [10, 15, 20, 25, 30, 40],
                         'select__score_func': [<function mutual_info_regression at 0x0000021E500F4B80>,
                                                <function f_regression at 0x0000021E500F7600>]})

In [75]:
grid_search3.best_score_

np.float64(0.4376373423415366)

In [76]:
grid_search3.best_params_

{'reg__alpha': 0.1,
 'reg__l1_ratio': 0,
 'select__k': 20,
 'select__score_func': <function sklearn.feature_selection._univariate_selection.f_regression(X, y, *, center=True, force_finite=True)>}

In [78]:
pipe4 = Pipeline([('select', SelectKBest()),('sc', StandardScaler()), ('reg', ElasticNet(random_state=42, max_iter=100000))], memory=None)
     

param_grid4 = {'select__score_func': [mutual_info_regression, f_regression],
              'select__k': [17, 18, 19, 20, 21, 22],
              'reg__alpha': [0.1, 0.15],
              'reg__l1_ratio': [0]
              }
     
grid_search4 = GridSearchCV(pipe4, param_grid4, cv=5)
grid_search4.fit(X_train, y_train)

c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.513e+02, tolerance: 2.249e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
c:\Users\lucas\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.597e+02, tolerance: 2.205e-01 Linear regression models with null weight for the l1 regularization term are more efficiently 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('select', SelectKBest()),
                                       ('sc', StandardScaler()),
                                       ('reg',
                                        ElasticNet(max_iter=100000,
                                                   random_state=42))]),
             param_grid={'reg__alpha': [0.1, 0.15], 'reg__l1_ratio': [0],
                         'select__k': [17, 18, 19, 20, 21, 22],
                         'select__score_func': [<function mutual_info_regression at 0x0000021E500F4B80>,
                                                <function f_regression at 0x0000021E500F7600>]})

In [ ]:
grid_search4.best_score_
# same parameters

np.float64(0.4376373423415366)

In [87]:
grid_search4.predict(X_test)

array([ 4.78948423,  7.23290997,  6.1916659 ,  7.42670512,  5.30845775,
        7.28052666,  8.62509844,  3.59715914,  8.48958002,  7.90166533,
        5.61697262,  8.92446667,  7.53665336,  2.92335117,  5.49714798,
        5.37082236,  7.11764732,  8.05986765,  6.64077145,  6.30089737,
        7.73645849,  5.72536547,  7.10543167,  7.68164235,  7.32927802,
        7.82706167,  4.82767591,  6.42490825, 10.45008969,  5.22271278,
        6.63746157,  6.00512413,  6.25833906,  5.19292324,  8.25911273,
        6.74013713,  7.24982402, 10.54755756,  6.00843785,  6.19243329,
        5.24259721,  6.46836537,  4.19421468,  6.47967481,  7.27560532,
        8.90011064,  5.16036584,  5.97093529,  4.59735341,  6.67287759,
        6.45478939,  4.37176685,  4.27444022,  4.77362734,  6.83888364,
        6.04316596,  6.16123898,  9.11219254,  8.06588864,  5.70972256,
        6.98154017,  8.73129571,  7.6217348 ,  7.39411105,  8.45397746,
        5.10987726,  6.92413446,  6.95055784,  3.76588865,  3.71

## Visualize result

In [112]:
grid_search4.best_estimator_

Pipeline(steps=[('select',
                 SelectKBest(k=20,
                             score_func=<function f_regression at 0x0000021E500F7600>)),
                ('sc', StandardScaler()),
                ('reg',
                 ElasticNet(alpha=0.1, l1_ratio=0, max_iter=100000,
                            random_state=42))])

In [122]:
selector = grid_search4.best_estimator_.named_steps['select']
selected_mask = selector.get_support()
chosen_features = X_train.columns[selected_mask]

In [128]:
regressor = grid_search4.best_estimator_.named_steps['reg']
coefs = regressor.coef_

## Predict my grade!

In [104]:
(my_X, my_y) = formatCode("input_data\mydata2.csv")
float(grid_search4.predict(my_X)[0])

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\lucas\AppData\Local\Temp\ipykernel_22444\232484384.py:1: SyntaxWarning: invalid escape sequence '\m'
  (my_X, my_y) = formatCode("input_data\mydata2.csv")


4.4970937875209955